In [2]:
import pandas as pd
from io import StringIO

csv_data = """period,total_revenue,net_income,gross_margin_pct,opex,cash_and_equivalents
2023,125000000,9000000,54.2,51000000,33000000
2024,137000000,12100000,55.6,52050000,41200000
"""

df = pd.read_csv(StringIO(csv_data))
df


,period,total_revenue,net_income,gross_margin_pct,opex,cash_and_equivalents
0,2023,125000000,9000000,54.2,51000000,33000000
1,2024,137000000,12100000,55.6,52050000,41200000


In [3]:
class DataStore:
    def __init__(self, df: pd.DataFrame):
        self.df = df.sort_values("period").reset_index(drop=True)

    def latest(self):
        return self.df.iloc[-1].to_dict()

    def latest_and_prev(self):
        latest = self.df.iloc[-1].to_dict()
        prev = self.df.iloc[-2].to_dict() if len(self.df) > 1 else None
        return latest, prev


In [4]:
def fmt_currency(x: float) -> str:
    return f"${x:,.0f}"

def fmt_pct(x: float) -> str:
    return f"{x:.1f}%"

def make_handlers(store):
    def total_revenue(_: str) -> str:
        latest = store.latest()
        return f"Total revenue: {fmt_currency(latest['total_revenue'])} (period {latest['period']})."

    def net_income_change(_: str) -> str:
        latest, prev = store.latest_and_prev()
        if not prev:
            return "Not enough data to calculate change."
        delta = latest['net_income'] - prev['net_income']
        pct = (delta / prev['net_income'] * 100) if prev['net_income'] else 0
        direction = "increased" if delta >= 0 else "decreased"
        return (f"Net income {direction} by {fmt_currency(abs(delta))} "
                f"({pct:.1f}%) from {prev['period']} to {latest['period']}.")

    def gross_margin(_: str) -> str:
        latest = store.latest()
        return f"Gross margin: {fmt_pct(latest['gross_margin_pct'])} (period {latest['period']})."

    def opex(_: str) -> str:
        latest = store.latest()
        return f"Operating expenses: {fmt_currency(latest['opex'])} (period {latest['period']})."

    def cash_on_hand(_: str) -> str:
        latest = store.latest()
        return f"Cash on hand: {fmt_currency(latest['cash_and_equivalents'])} (period {latest['period']})."

    return {
        "what is the total revenue?": total_revenue,
        "how has net income changed over the last year?": net_income_change,
        "what is the gross margin?": gross_margin,
        "what are operating expenses?": opex,
        "what is cash on hand?": cash_on_hand,
    }


In [5]:
class SimpleChatbot:
    def __init__(self, store):
        self.handlers = make_handlers(store)
        self.supported = list(self.handlers.keys())

    def normalize(self, q: str) -> str:
        return q.strip().lower()

    def reply(self, user_query: str) -> str:
        key = self.normalize(user_query)
        handler = self.handlers.get(key)
        if handler:
            return handler(user_query)
        return ("Sorry, I can only answer predefined queries. "
                f"Try: {', '.join(self.supported)}")


In [6]:
store = DataStore(df)
bot = SimpleChatbot(store)

# Example interactions
print(bot.reply("What is the total revenue?"))
print(bot.reply("How has net income changed over the last year?"))
print(bot.reply("What is cash on hand?"))
print(bot.reply("Who is the CEO?"))  # fallback example


Total revenue: $137,000,000 (period 2024.0).
Net income increased by $3,100,000 (34.4%) from 2023.0 to 2024.0.
Cash on hand: $41,200,000 (period 2024.0).
Sorry, I can only answer predefined queries. Try: what is the total revenue?, how has net income changed over the last year?, what is the gross margin?, what are operating expenses?, what is cash on hand?


In [ ]:
# Run this after you've created `bot`.
print("Financial Chatbot. Type a query or 'exit'.")
while True:
    try:
        user_q = input("> ")
    except EOFError:
        break
    if user_q.strip().lower() in {"exit", "quit"}:
        break
    print(bot.reply(user_q))


Financial Chatbot. Type a query or 'exit'.


>  What is the total revenue?


Total revenue: $137,000,000 (period 2024.0).
